In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df=pd.read_csv('/kaggle/input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv')

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
df=df.replace({'gender': {'M': 1, 'F': 0}, 'hsc_b': {'Central':0,'Others':1},'workex':{'Yes':1,'No':0},'specialisation':{'Mkt&HR':0,'Mkt&Fin':1},'status':{'Placed':1,'Not Placed':0}})

In [ ]:
df

In [ ]:
df=df.replace({'ssc_b': {'Others': 1, 'Central': 0}})

In [ ]:
df = pd.get_dummies(df)

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [ ]:
x=df.drop(['status','salary'],axis=1)
y=df['status']

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0,stratify=y)


In [ ]:
from sklearn.linear_model import LogisticRegression
algorithm1=LogisticRegression(random_state=0)

from sklearn.tree import DecisionTreeClassifier
algorithm2=DecisionTreeClassifier(random_state=0)

from sklearn.ensemble import RandomForestClassifier
algorithm3=RandomForestClassifier(random_state=0)

from xgboost import XGBClassifier
algorithm4=XGBClassifier(random_state=0)


from catboost import CatBoostClassifier
algorithm5=CatBoostClassifier(random_state=0)

algorithms=[algorithm1,algorithm2,algorithm3,algorithm4,algorithm5]


In [ ]:
from sklearn.model_selection import StratifiedKFold
stratifiedkfold=StratifiedKFold(n_splits=3)

from sklearn.model_selection import cross_val_score
for algorithm in algorithms:
  scores=cross_val_score(algorithm,x_train,y_train,cv=stratifiedkfold,scoring='roc_auc')
  score=scores.mean()
  name=algorithm.__class__.__name__
  print(f' average score : {score:.4f}  score : {scores}  {name}')


In [ ]:
def make_cm(matrix, columns):
    n = len(columns)

    act = ['correct data'] * n
    pred = ['prediction'] * n

    cm = pd.DataFrame(matrix, 
        columns=[pred, columns], index=[act, columns])
    return cm

In [ ]:
algorithm1.fit(x_train,y_train)
y_pred=algorithm1.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix
df_matrix=make_cm(
    confusion_matrix(y_test,y_pred),['Not placed','Plcaced'])
display(df_matrix)

In [ ]:
df['status'].value_counts()
# 1:placed 0:not placed

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision,recall,fscore,_=precision_recall_fscore_support(y_test,y_pred,average='binary')
print(f'precision: {precision:.4f}  recall: {recall: .4f}  Fscore:  {fscore:.4f}')


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
y_proba0=algorithm1.predict_proba(x_test)[:,1]

y0=y_proba0[y_test==0]
y1=y_proba0[y_test==1]

plt.figure(figsize=(6,6))
plt.title('prob')
sns.distplot(y1,kde=False,norm_hist=True,
             bins=50,color='b',label='placed')
sns.distplot(y0,kde=False,norm_hist=True,
             bins=50,color='k',label='Not plced')
plt.xlabel('prob')
plt.legend()
plt.show()


In [ ]:
def pred(algorithm1,x,thres):
  y_proba=algorithm1.predict_proba(x)
  y_proba1=y_proba[:,1]
  y_pred=(y_proba1>thres).astype(int)
  return y_pred

thres_list=np.arange(0.8,0,-0.05)

for thres in thres_list:
  y_pred=pred(algorithm1,x_test,thres)
  pred_sum=y_pred.sum()
  precision,recall,fscore,_=precision_recall_fscore_support(y_test,y_pred,average='binary')
  print(f'thres : {thres:.2f} pred : {pred_sum} precision: {precision:.4f}  recall: {recall: .4f}  Fscore:  {fscore:.4f})')
